In [51]:
import pandas as pd
import numpy as np
import requests as rq
import os
import chardet

pd.options.display.max_colwidth = 1000

In [52]:
DATA_PATH = '../data'

In [35]:
df_iegm = pd.read_csv(os.path.join(DATA_FOLDER,'iegm.csv'), delimiter=';', encoding='Windows-1252') 

/home/andrew/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [49]:
df_iegm.head()

,cod_ibge,municipio,porte,ano_exercicio,indice,cod_pergunta,texto_pergunta_genérica,cod_resposta,rotulo_resposta,valor_resposta
0,3500204,Adolfo,PP,2014.0,i-Amb,M05Q00100,"Sobre o Plano Municipal de Gestão Integrada de Resíduos Sólidos (PMGIRS) – Lei nº 12.305/2010 –, informe:",M05Q00100R00100,instrumento normativo publicado ou promulgado,instrumento normativo publicado ou promulgado
1,3501202,Álvares Florence,PP,2014.0,i-Amb,M05Q00100,"Sobre o Plano Municipal de Gestão Integrada de Resíduos Sólidos (PMGIRS) – Lei nº 12.305/2010 –, informe:",M05Q00100R00300,estão em outras fases de elaboração,está em outras fases de elaboração
2,3500303,Aguaí,M,2014.0,i-Amb,M05Q00100,"Sobre o Plano Municipal de Gestão Integrada de Resíduos Sólidos (PMGIRS) – Lei nº 12.305/2010 –, informe:",M05Q00100R00400,não realizaram o plano,não realizou o plano
3,3500402,Águas da Prata,P,2014.0,i-Amb,M05Q00100,"Sobre o Plano Municipal de Gestão Integrada de Resíduos Sólidos (PMGIRS) – Lei nº 12.305/2010 –, informe:",M05Q00100R00100,instrumento normativo publicado ou promulgado,instrumento normativo publicado ou promulgado
4,3500501,Águas de Lindóia,P,2014.0,i-Amb,M05Q00100,"Sobre o Plano Municipal de Gestão Integrada de Resíduos Sólidos (PMGIRS) – Lei nº 12.305/2010 –, informe:",M05Q00100R00300,estão em outras fases de elaboração,está em outras fases de elaboração


In [53]:
df_iegm.where(df_iegm['rotulo_resposta'] != df_iegm['valor_resposta']).dropna().head()

,cod_ibge,municipio,porte,ano_exercicio,indice,cod_pergunta,texto_pergunta_genérica,cod_resposta,rotulo_resposta,valor_resposta
1,3501202,Álvares Florence,PP,2014.0,i-Amb,M05Q00100,"Sobre o Plano Municipal de Gestão Integrada de Resíduos Sólidos (PMGIRS) – Lei nº 12.305/2010 –, informe:",M05Q00100R00300,estão em outras fases de elaboração,está em outras fases de elaboração
2,3500303,Aguaí,M,2014.0,i-Amb,M05Q00100,"Sobre o Plano Municipal de Gestão Integrada de Resíduos Sólidos (PMGIRS) – Lei nº 12.305/2010 –, informe:",M05Q00100R00400,não realizaram o plano,não realizou o plano
4,3500501,Águas de Lindóia,P,2014.0,i-Amb,M05Q00100,"Sobre o Plano Municipal de Gestão Integrada de Resíduos Sólidos (PMGIRS) – Lei nº 12.305/2010 –, informe:",M05Q00100R00300,estão em outras fases de elaboração,está em outras fases de elaboração
8,3500758,Alambari,P,2014.0,i-Amb,M05Q00100,"Sobre o Plano Municipal de Gestão Integrada de Resíduos Sólidos (PMGIRS) – Lei nº 12.305/2010 –, informe:",M05Q00100R00400,não realizaram o plano,não realizou o plano
10,3500907,Altair,PP,2014.0,i-Amb,M05Q00100,"Sobre o Plano Municipal de Gestão Integrada de Resíduos Sólidos (PMGIRS) – Lei nº 12.305/2010 –, informe:",M05Q00100R00300,estão em outras fases de elaboração,está em outras fases de elaboração


In [54]:
filter_cidade = df_iegm['indice'] == 'i-Cidade'

In [56]:
df_iegm.where(filter_cidade).dropna().head()

,cod_ibge,municipio,porte,ano_exercicio,indice,cod_pergunta,texto_pergunta_genérica,cod_resposta,rotulo_resposta,valor_resposta
17323,3500758,Alambari,P,2014.0,i-Cidade,M06Q00100,O município possui a Coordenadoria Municipal de Defesa Civil - COMDEC estruturada? (Lei nº 12.608/2012)?,M06Q00100R00200,estão em fase de estruturação,está em fase de estruturação
17324,3500204,Adolfo,PP,2014.0,i-Cidade,M06Q00100,O município possui a Coordenadoria Municipal de Defesa Civil - COMDEC estruturada? (Lei nº 12.608/2012)?,M06Q00100R00300,não possuem,não
17325,3500204,Adolfo,PP,2014.0,i-Cidade,M06Q00200,A prefeitura municipal possui local físico com sala e telefone para atendimento de ocorrências de Defesa Civil?,M06Q00200R00200,não possuem local físico,não
17326,3500204,Adolfo,PP,2014.0,i-Cidade,M06Q00300,O município está cadastrado no sistema da Defesa Civil estadual denominado SIDEC (Sistema Integrado de Defesa Civil)?,M06Q00300R00200,não possuem,não
17327,3500204,Adolfo,PP,2014.0,i-Cidade,M06Q00400,O município utiliza alguma forma de registro eletrônico para cadastramento de ocorrência de Defesa Civil?,M06Q00400R00200,não possuem,não


In [87]:
def get_df_from_api(tipo, municipio, exercicio, mes):
    r = rq.get('https://transparencia.tce.sp.gov.br/api/json/{}/{}/{}/{}'\
               .format(tipo, municipio, exercicio, mes))
    if r.status_code == 200:
        return pd.DataFrame(r.json())
    else:
        raise Exception('Bad Request')

In [88]:
df_adolfo = get_df_from_api('despesas','adolfo',2015,1)

In [91]:
df_adolfo.head()

,dt_emissao_despesa,evento,id_fornecedor,mes,nm_fornecedor,nr_empenho,orgao,vl_despesa
0,12/01/2015,Empenhado,CNPJ - PESSOA JURÍDICA - 01704233000138,Janeiro,FIORILLI SOC.CIVIL LTDA.,134-2015,PREFEITURA MUNICIPAL DE ADOLFO,"5869,68"
1,20/01/2015,Empenhado,IDENTIFICAÇÃO ESPECIAL - SEM CPF/CNPJ - 79,Janeiro,BANCO DO BRASIL,194-2015,PREFEITURA MUNICIPAL DE ADOLFO,"124,80"
2,13/01/2015,Empenhado,CNPJ - PESSOA JURÍDICA - 06291846000104,Janeiro,CONSTROESTE CONSTRUTORA E PARTICIPACOES LTDA,147-2015,PREFEITURA MUNICIPAL DE ADOLFO,"689,00"
3,07/01/2015,Empenhado,IDENTIFICAÇÃO ESPECIAL - SEM CPF/CNPJ - 2272,Janeiro,TRIBUNAL DE JUSTICA DO ESTADO DE SAO PAULO,114-2015,PREFEITURA MUNICIPAL DE ADOLFO,"14242,04"
4,20/01/2015,Empenhado,CNPJ - PESSOA JURÍDICA - 02558157000162,Janeiro,TELEFONICA BRASIL S.A,190-2015,PREFEITURA MUNICIPAL DE ADOLFO,"824,32"


In [90]:
df_adolfo['nm_fornecedor'].unique()

array(['FIORILLI SOC.CIVIL LTDA.', 'BANCO DO BRASIL',
       'CONSTROESTE CONSTRUTORA E PARTICIPACOES LTDA',
       'TRIBUNAL DE JUSTICA DO ESTADO DE SAO PAULO',
       'TELEFONICA BRASIL S.A', 'EMP. BRAS. DE CORREIOS E TELEGRAFOS',
       'CONFEDERACAO NACIONAL DE MUNICIPIOS', 'MELAINE REGINA MARTINEZ',
       'INST.NAC.PREV.SOCIAL', 'IMPRENSA OFICIAL DO ESTADO S/A - IMESP',
       'CAIXA ECONOMICA FEDERAL', 'NUTRICIONALE COM.DE ALIMENTOS LTDA.',
       'LARISSA BEJARANO RECHE', 'CAMILA MORAES BENATTI',
       'ECAD-ESCRIT.CENT.ARRECADACAO E DISTRIBUICAO',
       'CIA. NACIONAL DE ENERGIA ELETRICA',
       'JOSE NILTON DE OLIVEIRA DIVERSOES-ME', 'COP FAC MAQUINAS LTDA',
       'SABESP-COMP.SAN.BASICO ESTADO SP',
       'LABORATORIO ANAL.CLINICAS GALLI LTDA',
       'AUTO POSTO LARANJEIRAS DE MENDONCA LTDA EPP',
       'TEK CAMBIO JOSE BONIFACIO LTDA -ME', 'FRANCINALDO GOMES DA SILVA',
       'RAP APARECIDA COMERCIO DE MEDICAMENTO LTDA ME',
       'COMERCIAL CIRURGICA RIOCLARENSE LTDA'

http://dados.gov.br/noticia/casa-civil-abre-consulta-publica-sobre-inventario-de-dados-abertos